In [ ]:
import os
import urllib.request
from flask import Flask, request, render_template
from werkzeug.utils import secure_filename
import numpy as np
import tensorflow as tf
from keras.layers import TFSMLayer

app = Flask(__name__)
    
@app.route('/')
def upload_form():
    return render_template(
        'index.html')

@app.route('/', methods=['POST'])
def upload_image():
    file = request.files['file']
    filepath = os.path.join("static/uploads", secure_filename(file.filename))
    file.save(filepath)

    # Preprocess image
    imvar = tf.keras.preprocessing.image.load_img(filepath, target_size=(176, 176))
    imarr = tf.keras.preprocessing.image.img_to_array(imvar)
    imarr = np.expand_dims(imarr, axis=0)  # (1,176,176,3)

    # Load TF SavedModel
    model2 = TFSMLayer("model", call_endpoint="serving_default")

    # Call the model (returns a dict)
    outputs = model2(imarr)
    print("Model outputs:", outputs.keys())   # Debug — see what keys exist
    impred = list(outputs.values())[0].numpy()  # grab the first tensor

    # Get predicted class
    classcount = np.argmax(impred[0]) + 1  # +1 because dict starts at 1
    
    classdict = {
        1: "Mild Dementia",
        2: "Moderate Dementia",
        3: "No Dementia, Patient is Safe",
        4: "Very Mild Dementia"
    }
    print("Prediction:", classdict[classcount])

    return render_template(
        'index.html'
    )

if __name__ == "__main__":
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [24/Aug/2025 13:34:18] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Aug/2025 13:34:18] "GET /static/assets/github.png HTTP/1.1" 304 -
127.0.0.1 - - [24/Aug/2025 13:34:18] "GET /static/assets/linkedin.png HTTP/1.1" 304 -
127.0.0.1 - - [24/Aug/2025 13:34:18] "GET /static/assets/twitter.png HTTP/1.1" 304 -
127.0.0.1 - - [24/Aug/2025 13:34:18] "GET /static/assets/instagram.png HTTP/1.1" 304 -
127.0.0.1 - - [24/Aug/2025 13:34:18] "GET /static/assets/pexels-tima-miroshnichenko-6010766.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [24/Aug/2025 13:34:18] "GET /static/assets/pexels-tima-miroshnichenko-6010766.mp4 HTTP/1.1" 206 -
[2025-08-24 13:34:26,367] ERROR in app: Exception on / [POST]
Traceback (most recent call last):
  File "c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\flask\app.py", line 1473, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Use

mildDem2.jpg


127.0.0.1 - - [24/Aug/2025 13:35:02] "GET / HTTP/1.1" 200 -
